In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 24.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
import numpy as np
from datasets import load_dataset

In [3]:
dataset = load_dataset('cnn_dailymail', '1.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
noun_phrases = []

for i in range(0,1000):
  text = dataset['train'][i]['highlights']
  doc = nlp(text)
  # noun_phrases.extend([f'Doc{i+1}: '+chunk.text for chunk in doc.noun_chunks])
  noun_phrases.extend([chunk.text for chunk in doc.noun_chunks])


print(noun_phrases)

['Harry Potter star Daniel Radcliffe', '£20M fortune', 'he', 'Young actor', 'he', 'no plans', 'his cash', "Radcliffe's earnings", 'first five Potter films', 'trust fund', 'Mentally ill inmates', 'Miami', 'the "forgotten floor" Judge Steven Leifman', 'a result', '"avoidable felonies', 'CNN', 'facility', 'patient', 'I', 'the son', 'the president', 'Leifman', 'the system', 'he', 'change', 'I', 'I', 'driver', 'Man', 'pickup truck', 'half', 'he', 'face', 'I', 'a 30-, 35-foot free fall" Minnesota bridge', 'rush hour', 'procedure', 'none', 'spokesman', 'President', 'powers', 'vice president', 'Bush', 'routine colonoscopy', 'Camp David', 'NFL chief', 'Atlanta Falcons owner', "Michael Vick's conduct", 'NFL', 'Falcons', 'pay', 'Vick', 'funding', 'operation', 'he', 'Vick', 'federal court', 'future', 'NFL', 'Parents beam', 'pride', 'support', 'I', 'I', 'what', 'Burn center', 'U.S.', 'treatment', 'reconstructive surgeries', 'Dad', 'Anything', 'Youssif', 'workers', 'increased cost', 'living drive wo

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
context_matrix = tfidf.fit_transform(noun_phrases)

In [8]:
print(type(context_matrix))

<class 'scipy.sparse._csr.csr_matrix'>


In [9]:
tfidf.get_feature_names_out()

array(['000', '036', '047', ..., 'zuckerberg', 'zurich', 'zvonareva'],
      dtype=object)

In [10]:
def cosine_similarity_between_noun_phrases(np1, np2):
    np1_index = noun_phrases.index(np1)
    np2_index = noun_phrases.index(np2)
    cosine_sim = cosine_similarity(context_matrix[np1_index], context_matrix[np2_index])
    return cosine_sim[0][0]

# noun_phrase_1 = "Doc1: Harry Potter star Daniel Radcliffe"
# noun_phrase_2 = "Doc61: the Paris Masters"
# similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
# print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

# noun_phrase_1 = "Doc1: Harry Potter star Daniel Radcliffe"
# noun_phrase_2 = "Doc1: Radcliffe's earnings"
# similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
# print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

# noun_phrase_1 = "Doc1: Harry Potter star Daniel Radcliffe"
# noun_phrase_2 = "Doc1: £20M fortune"
# similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
# print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

noun_phrase_1 = "Harry Potter star Daniel Radcliffe"
noun_phrase_2 = "the Paris Masters"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

noun_phrase_1 = "Harry Potter star Daniel Radcliffe"
noun_phrase_2 = "Radcliffe's earnings"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

noun_phrase_1 = "Harry Potter star Daniel Radcliffe"
noun_phrase_2 = "£20M fortune"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

Cosine similarity between 'Harry Potter star Daniel Radcliffe' and 'the Paris Masters': 0.0
Cosine similarity between 'Harry Potter star Daniel Radcliffe' and 'Radcliffe's earnings': 0.3289249088959636
Cosine similarity between 'Harry Potter star Daniel Radcliffe' and '£20M fortune': 0.0
